# Trim a webnucleo network XML file relative to the naturally occurring isotopes.
The goal of this notebook is to trim a webnucleo network xml file relative to the naturally occurring isotopes.  Choose the range of isotopes either proton-rich or neutron-rich (or both) of the naturally occurring isotopes.  Don't worry if your notebook instance gets corrupted. Simply quit and download a new copy from GitHub, or start up a new version on Google Colab.

Begin by installing and importing the necessary python libraries

In [1]:
import sys, io, requests

!{sys.executable} -m pip install --quiet numpy
!{sys.executable} -m pip install --quiet wnutils

import numpy as np
import wnutils.xml as wx

Retrieve the JINA Reaclib nuclear data from [OSF](https://osf.io/kyhbs/).  Adjust XPath expressions for nuclear and reaction data, if desired.

In [2]:
nuc_xpath = ''
reac_xpath = ''

xml = wx.Xml(io.BytesIO(requests.get('https://osf.io/kyhbs/download').content))

nuc_data = xml.get_nuclide_data(nuc_xpath)
reac_data = xml.get_reaction_data(reac_xpath)

Retrieve the Solar System data.

In [3]:
solar = wx.Xml(io.BytesIO(requests.get('https://osf.io/j67qa/download').content))
solar_zone_data = solar.get_zone_data()

Create arrays of maximum and minimum A values for each Z.  Adjust values for Technetium (Z = 43) and Promethium (Z = 61).

In [4]:
a_min = np.full(84, 1000)
a_max = np.full(84, 0)

for tup in solar_zone_data['0']['mass fractions']:
    z = tup[1]
    a = tup[2]
    if z < 84:
        if a > a_max[z]:
            a_max[z] = a
        if a < a_min[z]:
            a_min[z] = a
            
a_min[43] = a_min[42]
a_max[43] = a_max[44]

a_min[61] = a_min[60]
a_max[61] = a_max[62]

Set *d_min* and *d_max*.  Exclude isotopes outside the range *A_min - d_min* and *A_max + d_max* for each element.  To include all nuclides, set *d_min* and *d_max* to large values (say, 1000).  The cell works from a copy of the original nuclide data, so after the full notebook is run, it is sufficient to change parameters in and execute this cell and those below.

In [5]:
d_min = 0
d_max = 10

nuc_data_work = nuc_data.copy()

for key, value in dict(nuc_data_work).items():
    z = value['z']
    a = value['a']
    if z > 83 or a < (a_min[z] - d_min) or a > (a_max[z] + d_max):
        del nuc_data_work[key]

Write out the new network XML file.  Adjust *my_output_xml* to your desired output XML name.  The file will be written in the working directory.  To download from Colab, click on the file tab to the left.

In [6]:
my_output_xml = 'my_output.xml'

new_xml = wx.New_Xml('nuclear_network')
new_xml.set_nuclide_data(nuc_data_work)
new_xml.set_reaction_data(reac_data)

new_xml.write(my_output_xml)